In [2]:
import torch
from torch import nn
from torch.nn import functional as F

In [3]:
model = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
model(X)

tensor([[-0.0240, -0.1219, -0.1023, -0.0984, -0.0585, -0.0441, -0.0194,  0.2405,
          0.1820,  0.0528],
        [-0.1372, -0.1390, -0.0409, -0.1883,  0.0394, -0.2026, -0.0270,  0.3177,
          0.2407,  0.1200]], grad_fn=<AddmmBackward0>)

**创建一个块**

1. 将输入数据作为其前向传播函数的参数。

2. 通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。

3. 计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。

4. 存储和访问前向传播计算所需的参数。

5. 根据需要初始化模型参数。

In [4]:
class MLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    
    def forward(self, X):
        hidden_layer = self.hidden(X)
        relu = F.relu(hidden_layer)
        output = self.output(relu)

        return output

$$ f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x} $$

In [5]:
class FixedHiddenMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.rand_weight = torch.rand((20, 20), requires_grad=True)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)

        while X.abs().sum() > 1:
            X /= 2
        return X.sum()


In [6]:
net = FixedHiddenMLP()
net(X)

tensor(0.3481, grad_fn=<SumBackward0>)

**参数管理...**

In [8]:
model = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))

X = torch.rand(size=(2, 4))
model(X)

tensor([[-0.5059],
        [-0.3327]], grad_fn=<AddmmBackward0>)

In [ ]:
def init_params(model):
    if type(model) == nn.Linear:
        nn.init.normal_(model.weight, mean=0, std=.01)
        nn.init.zeros_(model.bias)
model.apply(init_params)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)